In [198]:
import yfinance as yf

import pandas as pd
import numpy as np

from scipy.optimize import minimize
import statsmodels.api as sm

import matplotlib.pyplot as plt
import plotly.express as px

In [199]:
prices = yf.download(['VTI','VEA','ALTY','VWO','ICSH'])['Adj Close'].resample('W').last().fillna(method='ffill').dropna()
returns = prices.pct_change().fillna(0.0)
logReturns = np.log(prices/prices.shift(1))

px.histogram(returns).show()
px.histogram(logReturns).show()

[*********************100%***********************]  5 of 5 completed


# Functions

In [344]:
def get_risk(prices):
    return (prices / prices.shift(1) - 1).dropna().std() * np.sqrt(52)

def get_return(prices):
    return ((prices / prices.shift(1) - 1).dropna().mean() * 52)

def get_portfolio_risk(weights, returns):    
    return np.sqrt(np.dot(weights.T, np.dot(returns.cov(),weights)))

def get_portfolio_return(weights, returns):
    if type(returns) == pd.Series:
        return (weights * returns).sum()
    else:
        return returns.mul(weights, axis=1).sum(axis=1).mean()

def get_risk_adj_ret(weights, returns, riskFreeRate=0.0):    
    portRet = returns.mul(weights, axis=1).sum(axis=1)
    return (portRet.mean()-riskFreeRate)/portRet.std()

px.scatter(x=get_risk(prices).values, y=get_return(prices).values, 
           color=prices.columns, size=get_return(prices).values/get_risk(prices).values).show()

# Markowitz example

### Optimization example

In [201]:
def optimize(returns, symbols, target_return=None):
    init_guess = np.ones(len(symbols)) / len(symbols)
    
    bounds = ((0.0, 1.0),) * len(symbols)
    
    if target_return != None:
        constraints = ({'type':'eq','args': (returns, ), 
                        'fun': lambda x, returns: target_return-get_portfolio_return(weights=x,returns=returns)},
                       {'type':'eq','fun': lambda x: np.sum(x)-1})
    else:
        constraints = ({'type': 'eq', 'fun': lambda inputs: 1.0 - np.sum(inputs)})
    
    weights = minimize(get_portfolio_risk, init_guess,
                       args=(returns,), method='SLSQP',
                       options={'disp': False},
                       constraints=constraints,
                       bounds=bounds)
    return pd.Series(weights.x, index=returns.columns).round(4)

optimize(returns.drop('VTI', axis=1), returns.drop('VTI', axis=1).columns)

ALTY    0.0
ICSH    1.0
VEA     0.0
VWO     0.0
dtype: float64

### Random Portfolio example

In [314]:
def brute_opt(returns, tgtPortfolio = 'MaxSharpe', exp_returns=None, target_return=None, drawFrontier=False):
    portfoliosDF = pd.DataFrame(np.random.rand(100_000, len(returns.columns)), columns=returns.columns)
    portfoliosDF = portfoliosDF.div(portfoliosDF.sum(axis=1),axis=0)

    if type(exp_returns) == pd.Series:
        portfoliosDF['Mean'] = portfoliosDF[weights.columns].apply(get_portfolio_return, returns=exp_returns, axis=1) * 52
    else:
        portfoliosDF['Mean'] = portfoliosDF[weights.columns].apply(get_portfolio_return, returns=returns, axis=1) * 52
    portfoliosDF['STD'] = portfoliosDF[weights.columns].apply(get_portfolio_risk, returns=returns, axis=1) * np.sqrt(52)
    portfoliosDF['RiskAdjRet'] = portfoliosDF['Mean'] / portfoliosDF['STD']
    
    if drawFrontier:
        px.scatter(y=portfoliosDF['Mean'], x=portfoliosDF['STD'], color=portfoliosDF['RiskAdjRet']).show()
        
    if tgtPortfolio.lower() == 'maxsharpe': 
        return portfoliosDF.loc[portfoliosDF['RiskAdjRet'].idxmax()]
    elif tgtPortfolio.lower() == 'minvol': 
        return portfoliosDF.loc[portfoliosDF['STD'].idxmin()]
    elif tgtPortfolio.lower() == 'targetreturn' and target_return != None: 
        tgtPorts = portfoliosDF.loc[np.logical_and(portfoliosDF['Mean']>=target_return-0.001,
                                                   portfoliosDF['Mean']<=target_return+0.001)]        
        return tgtPorts.loc[tgtPorts['RiskAdjRet'].idxmax()]
    elif tgtPortfolio.lower() == 'all':
        return portfoliosDF
    
    pass

# CAPM example

In [203]:
betas = pd.Series(index=returns.drop('VTI', axis=1).columns, dtype=np.float64)

for a in returns.drop('VTI', axis=1).columns:
    y = returns[[a, 'VTI']].dropna()[a]
    X = returns[[a, 'VTI']].dropna()['VTI']
    
    model = sm.OLS(y,X)
    results = model.fit()
    betas.loc[a] = results.params['VTI']
    print(results.summary())

betas

                                 OLS Regression Results                                
Dep. Variable:                   ALTY   R-squared (uncentered):                   0.649
Model:                            OLS   Adj. R-squared (uncentered):              0.647
Method:                 Least Squares   F-statistic:                              509.6
Date:                Wed, 28 Oct 2020   Prob (F-statistic):                    1.21e-64
Time:                        22:00:01   Log-Likelihood:                          702.59
No. Observations:                 277   AIC:                                     -1403.
Df Residuals:                     276   BIC:                                     -1400.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

ALTY    1.023423
ICSH    0.034052
VEA     0.870095
VWO     0.827667
dtype: float64

In [316]:
def opt_CAPM(returns, market='VTI', tgtPortfolio='AboveMarket', target_return=None, graphFrontier=False):
    betas = pd.Series(index=returns.drop(market, axis=1).columns, dtype=np.float64)

    for a in returns.drop(market, axis=1).columns:
        y = returns[[a, market]].dropna()[a]
        X = returns[[a, market]].dropna()[market]

        model = sm.OLS(y,X)
        results = model.fit()
        betas.loc[a] = results.params[market]

    CAPM_expectedReturns = betas * returns[market].mean()
    
    allPorts = brute_opt(returns.drop(market, axis=1), tgtPortfolio = 'all', exp_returns=CAPM_expectedReturns)
    
    if graphFrontier:
        px.scatter(y=allPorts['Mean'].values,x=allPorts['STD'].values, color=allPorts['RiskAdjRet'].values).show()
    
    if tgtPortfolio.lower()=='abovemarket':
        if allPorts.loc[allPorts['Mean'] >= returns[market].mean()*52,'Mean'].count() > 0:
            return allPorts.loc[allPorts['Mean'] >= returns[market].mean()*52].sort_values('RiskAdjRet', 
                                                                                           ascending=False).iloc[0]
        else:
            return allPorts.loc[allPorts['Mean'] > allPorts['Mean'].mean()].sort_values('RiskAdjRet', 
                                                                                        ascending=False).iloc[0]
    elif tgtPortfolio.lower()=='minvol':
        return allPorts.loc[allPorts['STD'].idxmin()]
    elif tgtPortfolio.lower()=='maxsharpe':
        return allPorts.loc[allPorts['RiskAdjRet'].idxmax()]
    elif tgtPortfolio.lower()=='abovemarketminvol':
        if allPorts.loc[allPorts['Mean'] >= returns[market].mean()*52,'Mean'].count() > 0:
            return allPorts.loc[allPorts['Mean'] >= returns[market].mean()*52].sort_values('STD', ascending=True).iloc[0]
        else:
            return allPorts.loc[allPorts['Mean'] > allPorts['Mean'].mean()].sort_values('RiskAdjRet', 
                                                                                        ascending=False).iloc[0]
    elif target_return!=None:
        
        if allPorts.loc[allPorts['Mean'] >= target_return,'Mean'].count() > 0:
            return allPorts.loc[allPorts['Mean'] >= target_return].sort_values(by='RiskAdjRet', 
                                                                               ascending=False).iloc[0]
        else:
            return allPorts.loc[allPorts['Mean'] > allPorts['Mean'].mean()].sort_values('RiskAdjRet', 
                                                                                        ascending=False).iloc[0]
    elif tgtPortfolio.lower()=='all':
        return allPorts

In [317]:
allPorts = opt_CAPM(returns, market='VTI', tgtPortfolio='all')
px.scatter(y=allPorts['Mean'],x=allPorts['STD'], color=allPorts['RiskAdjRet']).show()

# Backtesting

In [295]:
backtestDF = pd.DataFrame(columns=list(['NAIVE','Markowitz_MeanVariance','Markowitz_CAPM',
                                       'Markowitz_MinVol','Markowitz_Target_Return',
                                       'CAPM_Target_Return','MARKET']),
                         index=returns.index[53:])

for c in backtestDF.columns:
    backtestDF[c] = 0.0
backtestDF['MARKET'] = returns.loc[backtestDF.index, 'VTI']

rebal_dates = returns.iloc[53:].resample('A').last().index.to_list()[:-1]
rebal_dates

[Timestamp('2016-12-31 00:00:00', freq='A-DEC'),
 Timestamp('2017-12-31 00:00:00', freq='A-DEC'),
 Timestamp('2018-12-31 00:00:00', freq='A-DEC'),
 Timestamp('2019-12-31 00:00:00', freq='A-DEC')]

### Initial weights and first results

In [322]:
ret = returns.iloc[:53]

init_weights = pd.DataFrame(index=backtestDF.drop('MARKET',axis=1).columns, 
                            columns=ret.drop('VTI',axis=1).columns, dtype=np.float64)

init_weights.loc['NAIVE'] = np.ones(len(ret.drop('VTI',axis=1).columns))/len(ret.drop('VTI',axis=1).columns)

# Markowitz portfolios first
portfolios = brute_opt(ret.drop('VTI',axis=1), tgtPortfolio = 'all')

init_weights.loc['Markowitz_MeanVariance'] = portfolios.loc[portfolios['RiskAdjRet'].idxmax()]
init_weights.loc['Markowitz_MinVol'] = portfolios.loc[portfolios['STD'].idxmin()]

if portfolios.loc[portfolios['Mean'].round(2)==0.07,'Mean'].count() > 0:
    tgtPort = portfolios.loc[portfolios['Mean'].round(2)==0.07].sort_values(by='RiskAdjRet', ascending=False).iloc[0]
    init_weights.loc['Markowitz_Target_Return'] = tgtPort
else:
    tgtPort = portfolios.loc[np.logical_and(portfolios['Mean'].round(2)>=0.06,
                                           portfolios['Mean'].round(2)<=0.08)].sort_values(by='RiskAdjRet', 
                                                                                           ascending=False).iloc[0]
    init_weights.loc['Markowitz_Target_Return'] = tgtPort

# Now the CAPM portfolios
portfolios = opt_CAPM(ret, market='VTI', tgtPortfolio='all')
init_weights.loc['Markowitz_CAPM'] = portfolios.loc[portfolios['RiskAdjRet'].idxmax()]

if portfolios['Mean'].max() < 0.07:
    portfolios = portfolios[portfolios['Mean']>=portfolios['Mean'].quantile(0.75)]
    init_weights.loc['CAPM_Target_Return'] = portfolios.sort_values(by='RiskAdjRet', ascending=False).iloc[0]
else:
    init_weights.loc['CAPM_Target_Return'] = portfolios[portfolios['Mean'] >=0.07].sort_values(by='RiskAdjRet', 
                                                                                               ascending=False).iloc[0]

init_weights

,ALTY,ICSH,VEA,VWO
NAIVE,0.250000,0.250000,0.250000,0.250000
Markowitz_MeanVariance,0.152439,0.830625,0.006270,0.010666
Markowitz_CAPM,0.086240,0.032518,0.766552,0.114690
Markowitz_MinVol,0.009065,0.945676,0.044012,0.001248
Markowitz_Target_Return,0.683335,0.304008,0.009539,0.003119
CAPM_Target_Return,0.086240,0.032518,0.766552,0.114690


In [323]:
stDate = returns.index[53]
enDate = rebal_dates[0]

for p in backtestDF.drop('MARKET', axis=1).columns:
    backtestDF.loc[stDate:enDate,p] = returns.loc[stDate:enDate,init_weights.columns].mul(init_weights.loc[p]).sum(axis=1)

px.line(backtestDF.add(1).cumprod().mul(100).loc[stDate:enDate]).show()

### Now we repeat the process for annual rebalancing

In [326]:
for rb in rebal_dates:
    if rebal_dates.index(rb) == 0:
        ret = returns.loc[:rb]
    else:
        ret = returns.loc[rebal_dates[rebal_dates.index(rb)-1]:rb]

    weights = pd.DataFrame(index=backtestDF.drop('MARKET',axis=1).columns, 
                           columns=ret.drop('VTI',axis=1).columns, dtype=np.float64)

    weights.loc['NAIVE'] = np.ones(len(ret.drop('VTI',axis=1).columns))/len(ret.drop('VTI',axis=1).columns)

    # Markowitz portfolios first
    portfolios = brute_opt(ret.drop('VTI',axis=1), tgtPortfolio = 'all')

    weights.loc['Markowitz_MeanVariance'] = portfolios.loc[portfolios['RiskAdjRet'].idxmax()]
    weights.loc['Markowitz_MinVol'] = portfolios.loc[portfolios['STD'].idxmin()]

    if portfolios.loc[portfolios['Mean'].round(2)==0.07,'Mean'].count() > 0:
        tgtPort = portfolios.loc[portfolios['Mean'].round(2)==0.07].sort_values(by='RiskAdjRet', ascending=False).iloc[0]
        weights.loc['Markowitz_Target_Return'] = tgtPort
    else:
        tgtPort = portfolios.loc[portfolios['Mean']>portfolios['Mean'].quantile(0.75)].sort_values(by='RiskAdjRet', 
                                                                                                   ascending=False).iloc[0]
        weights.loc['Markowitz_Target_Return'] = tgtPort

    # Now the CAPM portfolios
    weights.loc['Markowitz_CAPM'] = opt_CAPM(ret, market='VTI', tgtPortfolio='MaxSharpe')
    weights.loc['CAPM_Target_Return'] = opt_CAPM(ret, market='VTI', target_return=0.07)

    stDate = rb
    if rebal_dates.index(rb) == len(rebal_dates)-1:
        enDate = returns.index.to_list()[-1]
    else:
        enDate = rebal_dates[rebal_dates.index(rb) + 1]

    for p in backtestDF.drop('MARKET', axis=1).columns:
        backtestDF.loc[stDate:enDate,p] = returns.loc[stDate:enDate,weights.columns].mul(weights.loc[p]).sum(axis=1)
    
    px.line(backtestDF.loc[:enDate].add(1).cumprod().mul(100)).show()
    
backtestDF.describe()

,NAIVE,Markowitz_MeanVariance,Markowitz_CAPM,Markowitz_MinVol,Markowitz_Target_Return,CAPM_Target_Return,MARKET
count,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000,224.000000
mean,0.000948,0.000480,0.001521,0.000455,0.000510,0.000854,0.002560
std,0.019660,0.003742,0.008005,0.003618,0.009356,0.022421,0.026228
min,-0.125120,-0.034815,-0.029486,-0.034815,-0.063017,-0.141496,-0.152578
25%,-0.005641,-0.000320,-0.001497,-0.000127,-0.001762,-0.004140,-0.005459
50%,0.001527,0.000587,0.000888,0.000521,0.000696,0.001637,0.004075
75%,0.008397,0.001483,0.004541,0.001271,0.003250,0.007251,0.014196
max,0.103931,0.023103,0.029281,0.023103,0.052238,0.133396,0.129519


In [329]:
portfolioIndexes = backtestDF.add(1).cumprod().mul(100)
px.line(portfolioIndexes).show()

In [331]:
portfolioIndexes.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 224 entries, 2016-07-24 to 2020-11-01
Freq: W-SUN
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NAIVE                    224 non-null    float64
 1   Markowitz_MeanVariance   224 non-null    float64
 2   Markowitz_CAPM           224 non-null    float64
 3   Markowitz_MinVol         224 non-null    float64
 4   Markowitz_Target_Return  224 non-null    float64
 5   CAPM_Target_Return       224 non-null    float64
 6   MARKET                   224 non-null    float64
dtypes: float64(7)
memory usage: 24.0 KB


In [343]:
stats = pd.DataFrame(index=portfolioIndexes.columns)
stats['DrawDown'] = (portfolioIndexes/portfolioIndexes.rolling(52).max()-1).min()
stats['AVG Ann. Ret'] = (portfolioIndexes/portfolioIndexes.shift(52)-1).mean()
stats['AVG Ann. Risk'] = (portfolioIndexes/portfolioIndexes.shift(52)-1).std()
stats['Sharpe'] = stats['AVG Ann. Ret']/stats['AVG Ann. Risk']
stats['Total Ret.'] = portfolioIndexes.iloc[-1]/portfolioIndexes.iloc[0]-1
stats['Ann. Tot. Ret.'] = np.power(stats['Total Ret.'].add(1), 52/len(portfolioIndexes.index))-1

stats

,DrawDown,AVG Ann. Ret,AVG Ann. Risk,Sharpe,Total Ret.,Ann. Tot. Ret.
NAIVE,-0.295697,0.036528,0.083604,0.436910,0.173498,0.037839
Markowitz_MeanVariance,-0.060371,0.025045,0.010406,2.406804,0.106455,0.023762
Markowitz_CAPM,-0.044419,0.083895,0.056353,1.488746,0.391760,0.079761
Markowitz_MinVol,-0.060371,0.024690,0.009883,2.498217,0.104163,0.023269
Markowitz_Target_Return,-0.126546,0.019809,0.039060,0.507143,0.092241,0.020694
CAPM_Target_Return,-0.326304,0.032490,0.120381,0.269895,0.140208,0.030928
MARKET,-0.329121,0.119705,0.091943,1.301950,0.628571,0.119875
